In [2]:
#Packages

import math
import numpy as np
import scipy as sp
import numpy.random as npr  
import scipy.stats as scs
import matplotlib.pyplot as plt
import numpy.random as npr
import pandas as pd
import seaborn as sns
import statistics


#seaborn.set_style("ticks")

from numpy import meshgrid, sqrt, diff
from scipy import inf, pi, exp, linspace, zeros, real, imag, array, log
from scipy.stats import norm
from scipy.special import p_roots

In [3]:
# SVCJ parameters
mu      = 0.042
r       = mu
mu_y    = -0.0492
sigma_y = 2.061
l       = 0.0515
alpha   = 0.0102
beta    = -0.188
rho     = 0.275
sigma_v = 0.007
rho_j   = 0#-0.210
mu_v    = 0.709
v0      = 0.19**2 
kappa   = 1-beta
theta   = alpha / kappa

In [4]:
npr.seed(12345)
dt      = 1/360.0# dt
m       = int(360.0 * (1/dt)/90.) # time horizon in days
n       = 10000*3

#for trials
#dt = 1/10
#n= 1000
#m = int(10*(1/dt)/10)


T      = m * dt
t      = np.arange(0,T+dt, dt)
ttm    = T- np.arange(0, m+1, 1)/m

#Hedge 
hestondelta     = np.zeros([n,m+1])
ttmmatrix       = np.insert(ttm, 0, 0., axis=0)
ttmmatrix       = T-np.tile(ttm, (n, 1))
ttmmatrix.shape

#for trials
#dt = 1/10
#n= 1000
#m = int(10*(1/dt)/10)

(30000, 1441)

In [5]:
#Heston model parameters

kappa_heston = 67.521
theta_heston = 0.006
sigmav_heston = 0.920
rho_heston   = -0.994
v0_heston    = 3.406
r_heston     = mu



In [6]:
w      = npr.standard_normal([n,m])
w2     = rho * w + sp.sqrt(1-rho**2) * npr.standard_normal([n,m])
z_v    = npr.exponential(mu_v, [n,m])
z_y    = npr.standard_normal([n,m]) * sigma_y + mu_y + rho_j * z_v
dj     = npr.binomial(1, l * dt, size=[n,m])
s      = np.zeros([n,m+1])
v      = np.zeros([n,m+1])

In [7]:
s0     = 6500
k      = 6500
s[:,0] = s0 # initial CRIX level, p. 20
v[:,0] = v0

In [8]:
for i in range(1,m+1):
    v[:,i] = v[:,i-1] + kappa * (theta - np.maximum(0,v[:,i-1])) * dt + sigma_v * sp.sqrt(np.maximum(0,v[:,i-1])) * w2[:,i-1] + z_v[:,i-1] * dj[:,i-1]
    s[:,i] = s[:,i-1] * (1 + (r - l * (mu_y + rho_j * mu_v)) * dt + sp.sqrt(v[:,i-1] * dt) * w[:,i-1]) + z_v[:,i-1] * dj[:,i-1]

/Users/lilimatic/anaconda3/envs/lilisthesis/lib/python3.7/site-packages/ipykernel_launcher.py:3: ComplexWarning: Casting complex values to real discards the imaginary part
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
k = 6500*0.95

In [10]:
# Option pricing 
cp    = np.exp(-mu * m * dt) * np.maximum(s[:,-1]-k,0).mean()
cp

1595.812000498775

In [11]:
def integrate(func):

    pts = 20

    [phi, w] = p_roots(pts)

    a = 0
    b = 500

    x = (b - a) * phi / 2 + (a + b) / 2

    smatrix2   = np.resize(s, (pts, n, m+1))
    ttmmatrix2 = np.resize(ttmmatrix, (pts, n, m+1))
    xmatrix2   = np.transpose(np.resize(x, (m+1, n, pts)))

    return np.transpose((b - a) / 2 * np.sum((w * np.transpose(func(smatrix2, ttmmatrix2, xmatrix2))), axis=2))

In [12]:
def plot(output, i):

    fig, ax1 = plt.subplots()

    color = 'tab:red'
    ax1.set_xlabel('time (s)')
    ax1.set_ylabel('spot', color=color)
    ax1.plot(s[i,0:-1], color=color)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:blue'
    ax2.set_ylabel('delta', color=color)  # we already handled the x-label with ax1
    ax2.plot(output[i,0:-1], color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.savefig('DeltaPlot'+str(i),transparent=T)
    plt.show()

\begin{equation}
\begin{aligned} g &=\frac{b_{j}-\rho \sigma \phi i+d}{b_{j}-\rho \sigma \phi i-d} \\ d &=\sqrt{\left(\rho \sigma \phi i-b_{j}\right)^{2}-\sigma^{2}\left(2 u_{j} \phi i-\phi^{2}\right)} \end{aligned}
\end{equation}

## 1. Program 

\begin{equation}
\begin{aligned} 
f_{j}\left(x, V_{t}, T, \phi\right) &=\exp \left\{C(T-t, \phi)+D(T-t, \phi) V_{t}+i \phi x\right\} \\ 
C(T-t, \phi) &=r \phi i r+\frac{a}{\sigma^{2}}\left[\left(b_{j}-\rho \sigma \phi i+d\right) \tau-2 \ln \left(\frac{1-g e^{d r}}{1-g}\right)\right] \\ D(T-t, \phi) &=\frac{b_{j}-\rho \sigma \phi i+d}{\sigma^{2}}\left(\frac{1-e^{d r}}{1-g e^{d r}}\right) \end{aligned}
\end{equation}

## 2.Program 

\begin{equation}
\begin{aligned} P_{j}\left(x, V_{t}, T, K\right) &=\frac{1}{2}+\frac{1}{\pi} \int_{0}^{\infty} \operatorname{Re}\left(\frac{e^{-i \phi \ln (K)} f_{j}\left(x, V_{t}, T, \phi\right)}{i \phi}\right) d \phi \\ x &=\ln \left(S_{t}\right) \end{aligned}
\end{equation}







In [13]:
# inner integral function 

def funkcija(s, ttm, phi, type):
    if type == 1:
        u = 0.5
        b = kappa_heston - rho_heston*sigmav_heston
    else:
        u = -0.5
        b = kappa_heston
        
    a = kappa_heston*theta_heston
    x = sp.log(s)
    
    d = sp.sqrt((rho_heston*sigmav_heston*phi*1j-b)**2-(sigmav_heston**2)*(2*u*phi*1j-phi**2))
    g = (b-rho_heston*sigmav_heston*phi*1j + d)/(b-rho_heston*sigmav_heston*phi*1j - d)
    
    C = r*phi*1j*ttm + (a/(sigmav_heston**2)) *((b- rho_heston*sigmav_heston*phi*1j + d)*ttm -2*sp.log((1-g*sp.exp(d*ttm))/(1-g)));

    D = (b-rho_heston*sigmav_heston*phi*1j + d)/(sigmav_heston**2)*((1-sp.exp(d*ttm))/ (1-g*sp.exp(d*ttm)));
    f = sp.exp(C + D*v0_heston + 1j*phi*x)
    
    return f

In [ ]:
def integrand(s, ttm, phi):
    return (sp.exp(-1j*phi*sp.log(k))*funkcija(s, ttm, phi, 1)/(1j*phi)).real

P1 = .5 + integrate(integrand) / sp.pi
    
plot(P1, 20)

In [ ]:
# inner integral function 

def innerfunc1(s, ttm, phi):
    return(sp.exp(-1j*phi*sp.log(k))*funkcija(s, ttm, phi, 1)/(s)).real

In [ ]:
def innerfunc2(s, ttm, phi):
    return(sp.exp(-1j*phi*sp.log(k))*funkcija(s, ttm, phi, 2)/(s)).real

delta = P1 + s * integrate(innerfunc1) / sp.pi - k * integrate(innerfunc2) / sp.pi
    
plot(delta, 20)

In [ ]:
# initial setup of hedge; this is the same for all simulation scenarios
t = 0;
d = delta[0,0]
s_pos = d*s[0,0] # hedge in stock
b_pos = cp - s_pos # borrow remainder from bank account
[cp, d, s_pos, b_pos, (s_pos + b_pos)]

b_pos0 = b_pos
s_pos0 = s_pos
d0 = d

PnL = np.zeros(n);

# daily rebalancing

for i in range(1,m):
    ttm = T - i*dt
    pos = d * s[:,i] + b_pos * np.exp(r*dt) # current position value
    d = delta[:,i] # new delta
    s_pos = d * s[:,i]
    b_pos = pos - s_pos # make strategy self-financing any surplus or funds needed go through the bank account
    # final day / payoff
    
PnL = (- np.maximum(s[:,-1]-6500,0) + d * s[:,i] + b_pos * np.exp(r*dt))/cp

In [ ]:
c, bins, patches = plt.hist(x=PnL, bins=1000, color='b',alpha=0.7, rwidth=0.7)
plt.xlabel('Relative profit')
plt.ylabel('Frequency')
#plt.title('Black Scholes Delta Hedge Relative Profit and Loss for a spot of 6500 and a strike of 6500')
maxfreq = c.max()
# Set a clean upper y-axis limit.
#plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)
plt.xlim(xmin = -2.5, xmax=1)
plt.savefig('PnLBSDeltaHedgeATMPNL',transparent=T)
plt.show()

In [ ]:
hedgeerror = [100*statistics.stdev(PnL)/cp]
hedgeerror

In [ ]:
a = np.array(PnL) 
quantile = [0.001, 0.01, 0.05,0.1,0.25,0.50,0.75,0.90,0.95, 0.99,0.999]
quantilesBS90 = np.zeros(len(quantile))
for i in range(len(quantile)):
    quantilesBS90[i] = np.quantile(a,quantile[i])
quantilesBS90

In [ ]:
BS90 = [round(x,3) for x in list(quantilesBS90)]

In [ ]:
std90 = round(statistics.stdev(PnL),3)
skew  = round(scs.skew(PnL),3)
kurt  = round(scs.kurtosis(PnL),3)

In [ ]:
[std90, 
skew,  
kurt] 

In [ ]:
kurt  = round(scs.kurtosis(PnL),3)

In [ ]:
np.quantile(PnL,0.5)

In [ ]:
plot(delta, 20)
plot(delta, 1000)

In [ ]:
BS90

In [ ]:
[-6.072,
 -1.679,
 -0.184,
 -0.064,
 0.121,
 0.323,
 0.508,
 0.644,
 0.714,
 0.875,
 3.54]

In [ ]:
array([-4.88657545, -1.47178906, -0.15053428, -0.04047356,  0.13359705,
        0.3233244 ,  0.50038086,  0.62334402,  0.68679899,  0.818698  ,
        4.81889841])

In [ ]:
delta.shape


In [ ]:
BS90

In [ ]:
#180[3.116, -75.567, 672.417]